In [ ]:
!git clone https://github.com/sarthak7509/ConversationalAi.git

Cloning into 'ConversationalAi'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 34 (delta 4), reused 29 (delta 2), pack-reused 0
Unpacking objects: 100% (34/34), done.


In [ ]:
!pip install tensorflow-datasets

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import pandas as pd
import os
import io
import time
import pickle

In [3]:
path_to_dataset  = "WikiQA-train.tsv"

In [4]:
df = pd.read_csv(path_to_dataset,sep='\t', header=0)

In [5]:
#preprocessing
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
    sentence = sentence.strip()
    # adding a start and an end token to the sentence
    # sentence = '<start> ' + sentence + ' <end>'
    return sentence

In [6]:
question = df["Question"]
answer = df['Sentence']

In [8]:
questions=[]
answers=[]
for ques,ans in zip(question,answer):
    questions.append(preprocess_sentence(ques))
    answers.append(preprocess_sentence(ans))

In [9]:
print(f"questions {questions[10]}")
print(f"answere {answers[10]}")

questions how are the directions of the velocity and force vectors related in a circular motion
answere since the object s velocity vector is constantly changing direction , the moving object is undergoing acceleration by a centripetal force in the direction of the center of rotation .


In [10]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)

# Define start and end token to indicate the start and end of a sentence
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

VOCAB_SIZE = tokenizer.vocab_size+2

In [11]:
print(f"encoded question example {tokenizer.encode(questions[10])}")

encoded question example [15, 14, 1, 6204, 8055, 3, 1, 2476, 3518, 355, 5, 890, 304, 5566, 9, 605, 6, 8, 4805, 63, 2820, 102]


In [13]:
with open('Wikiqatokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
#setting maximum length of a sentence
MAX_LENGTH=100
def tokennize_filter(inputs,outputs):
    tokenized_inputs,tokenized_output = [],[]
    for (sen1,sen2) in zip(inputs,outputs):
        sen1 = START_TOKEN + tokenizer.encode(sen1) + END_TOKEN
        sen2 = START_TOKEN + tokenizer.encode(sen2) + END_TOKEN
    if len(sen1)<=MAX_LENGTH and len(sen2) <=MAX_LENGTH:
        tokenized_inputs.append(sen1)
        tokenized_output.append(sen2)
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_inputs,maxlen=MAX_LENGTH,padding='post')
    tokenized_output = tf.keras.preprocessing.sequence.pad_sequences(tokenized_output,maxlen=MAX_LENGTH,padding='post')
    return tokenized_inputs,tokenized_output

In [15]:
questions,answers = tokennize_filter(questions,answers)

In [16]:
print(f'Vocab Size {VOCAB_SIZE}')
print(f'Number of samples {len(questions)}')

Vocab Size 8281
Number of samples 1


In [17]:
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs':answers[:,:-1]
    },
    {
        'outputs':answers[:,1:]
    },
))

In [18]:
BATCH_SIZE=64
BUFFER_SIZE=20000

In [19]:
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).cache().prefetch(tf.data.experimental.AUTOTUNE)

In [20]:
def attention(query,key,value,mask):
    qk_matmul = tf.matmul(query,key,transpose_b=True)
    depth= tf.cast(tf.shape(key)[-1],tf.float32)
    logits = qk_matmul/tf.math.sqrt(depth)

    if mask is not None:
        logits += (mask *-1e9)
    attention_weight = tf.nn.softmax(logits,axis=-1)
    output = tf.matmul(attention_weight,value)

    return output

In [21]:
class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(
            inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # linear layers
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # split heads
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # scaled dot-product attention
        scaled_attention = attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # concatenation of heads
        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # final linear layer
        outputs = self.dense(concat_attention)

        return outputs

In [22]:

def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]

In [23]:
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

In [24]:
#creating postional encoding for the layer to get the word postion
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self,postion,d_model):
        super(PositionalEncoding,self).__init__()
        self.pos_encoding = self.postional_encoding(postion,d_model)
    def get_angles(self,postion,i,d_model):
        angles = 1/tf.pow(10000,(2*(i//2))/tf.cast(d_model,dtype=tf.float32))
        return postion*angles
    def postional_encoding(self,postion,d_model):
        angle_rads = self.get_angles(
            postion = tf.range(postion,dtype=tf.float32)[:,tf.newaxis],
            i = tf.range(d_model,dtype=tf.float32)[tf.newaxis,:],
            d_model = d_model
        )
        # apply sin to even index in the array
        sines = tf.math.sin(angle_rads[:, 0::2])
        # apply cos to odd index in the array
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self,inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [25]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
    attention = tf.keras.layers.Dropout(rate=0.3)(attention)
    attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [26]:
def encoder(vocab_size,num_layers,units,d_model,num_heads,dropout,name='encoder'):
    inputs = tf.keras.Input(shape=(None,),name='inputs')
    padding_mask = tf.keras.Input(shape=(1,1,None),name='padding_mask')

    embeddings = tf.keras.layers.Embedding(vocab_size,d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model,tf.float32))
    embeddings = PositionalEncoding(vocab_size,d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)
    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])
    return tf.keras.Model(
      inputs=[inputs,padding_mask],outputs=outputs,name=name
      )

In [27]:

def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")

    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")

    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
    attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

    attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [28]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [29]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

    enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)
    # mask the future tokens for decoder inputs at the 1st attention block
    look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)
    # mask the encoder outputs for the 2nd attention block
    dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

    enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
    )(inputs=[inputs, enc_padding_mask])

    dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
    )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [30]:
#choosing hyperparameter
NUM_LAYERS=2
D_MODEL=256
NUM_HEADS=8
UNITS = 512
DROPOUT=0.1

model = transformer(vocab_size=VOCAB_SIZE,num_layers=NUM_LAYERS,units=UNITS,d_model=D_MODEL,num_heads=NUM_HEADS,dropout=DROPOUT)

In [31]:
model.load_weights("../model_weight_WkiiQNa/model2.h5")

In [36]:
model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 256)    3174144     inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

In [37]:
def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

    loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

In [38]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [39]:

learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    # ensure labels have shape (batch_size, MAX_LENGTH - 1)
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [ ]:
EPOCHS = 20

model.fit(dataset, epochs=EPOCHS)

Epoch 1/20
318/318 [==============================] - 54s 141ms/step - loss: 0.5474 - accuracy: 0.1870
Epoch 2/20
318/318 [==============================] - 45s 142ms/step - loss: 0.5035 - accuracy: 0.1945
Epoch 3/20
318/318 [==============================] - 45s 143ms/step - loss: 0.4720 - accuracy: 0.1997
Epoch 4/20
318/318 [==============================] - 46s 144ms/step - loss: 0.4434 - accuracy: 0.2049
Epoch 5/20
318/318 [==============================] - 46s 144ms/step - loss: 0.4208 - accuracy: 0.2094
Epoch 6/20
318/318 [==============================] - 46s 145ms/step - loss: 0.3988 - accuracy: 0.2134
Epoch 7/20
318/318 [==============================] - 46s 146ms/step - loss: 0.3797 - accuracy: 0.2172
Epoch 8/20
318/318 [==============================] - 46s 143ms/step - loss: 0.3628 - accuracy: 0.2204
Epoch 9/20
318/318 [==============================] - 46s 145ms/step - loss: 0.3478 - accuracy: 0.2237
Epoch 10/20
318/318 [==============================] - 46s 143ms/step - l

In [33]:
def evaluate(sentence):
    sentence = preprocess_sentence(sentence)

    sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

    output = tf.expand_dims(START_TOKEN, 0)

    for i in range(MAX_LENGTH):
        predictions = model(inputs=[sentence, output], training=False)

        # select the last word from the seq_len dimension
        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # return the result if the predicted_id is equal to the end token
        if tf.equal(predicted_id, END_TOKEN[0]):
            break

    # concatenated the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0)


def predict(sentence):
    prediction = evaluate(sentence)

    predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

    print('Input: {}'.format(sentence))
    print('Output: {}'.format(predicted_sentence))

    return predicted_sentence

In [40]:
output=predict("what are the glaciers")

Input: what are the glaciers
Output: the 


In [ ]:
questionsa=[]
answersa=[]
for ques,ans in zip(question,answer):
    questionsa.append(preprocess_sentence(ques))
    answersa.append(preprocess_sentence(ans))

In [ ]:
model.save_weights("/content/gdrive/MyDrive/wkiiQb/model2")